In [ ]:
!pip install requests pandas numpy matplotlib seaborn nltk emoji scikit-learn==1.3.2 imbalanced-learn==0.11.0 wordcloud

import os
import requests
import csv
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns


import os
import re
import emoji

from collections import Counter, defaultdict
from nltk.corpus import stopwords
from string import punctuation
import nltk
nltk.download('stopwords')
sw = stopwords.words("english")
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('omw-1.4')
import string
from nltk.tokenize import word_tokenize

import matplotlib.pyplot as plt
import seaborn as sns
import html 
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.pipeline import make_pipeline, Pipeline
from collections import Counter
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
from sklearn.metrics import ConfusionMatrixDisplay
from sklearn.model_selection import cross_val_score
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV
from sklearn.svm import LinearSVC
from sklearn.dummy import DummyClassifier
from sklearn.metrics import confusion_matrix, accuracy_score, ConfusionMatrixDisplay, classification_report
from sklearn.model_selection import train_test_split, cross_val_score

import warnings
warnings.filterwarnings('ignore')
warnings.simplefilter(action='ignore', category=FutureWarning)

## Data Loading 

In [ ]:
#loading data 
df = pd.read_csv("/kaggle/input/dataset/fake_job_postings.csv")
df.head(5)

## Exploratory Data Analysis(EDA)

In this section we will be exploring our data by looking at the various aspects such as checking null values, duplicates, class imbalance, data types. 

### Checking Null Values

In [ ]:
#to checking null values
df.isna().sum()

In [ ]:
#checking data shape
df.shape

### Information on Missing Values

In [ ]:
#sustituting null values with na 
df_na =df.fillna('na') 

real=df_na[df_na.fraudulent==0]
fake=df_na[df_na.fraudulent==1]



In [ ]:
#cheching the rates of na 
na_rates=pd.DataFrame([col, len(real.loc[real[col]=='na'])/len(real[col]), len(fake.loc[fake[col]=='na'])/len(fake[col]) ] for col in df.columns)

na_rates.columns=['column','real_na_rates','fake_na_rates']


In [ ]:
na_rates

In [ ]:
#z-test on null values 
from statsmodels.stats.proportion import proportions_ztest
for col in df.columns:
    counts=np.array([len(real.loc[real[col]=='na']), len(fake.loc[fake[col]=='na'])])
    nobs=np.array([len(real[col]), len(fake[col])])
    if (counts.sum() !=0) and (nobs.sum() !=0):
        na_rates.loc[na_rates.column==col,'zstat'], na_rates.loc[na_rates.column==col,'p_value']=proportions_ztest(count=counts, nobs=nobs,  alternative='two-sided')

In [ ]:
na_rates['significant_diff']=na_rates['p_value']<0.005
na_rates

**Observation**

The Z-tests indicate that differences of na-rates among real and fake job-postings are mostly statistically significant, except in 'location', 'department', 'benefits' and 'function'. The difference is largest in company profile, where 68% of fake ads do not have a company profile, but only 16.1% of real ads omit company profiles.

### Checking Data Types 

In [ ]:
#obtaining data types
df.dtypes

### Checking Duplicates 

In [ ]:
#counting duplicate value
df.duplicated().value_counts()

**Observation:** 

No duplicates identified. 

In [ ]:
# statististics of numeric data 
df.describe()

**From describe() fuctution we can see our data has 4 numerical columns named as job_id, telecommuting, has_company_logo and has_questions.We can  remove these columns as they are of no use in text classification problems. We can see another numerical feature 'fraudulent' but here this columns will be used as our target variable in which the models will be trained and predicted.**

### Checking Class Imbalance 

In [ ]:
# Now lets see how many jobs posted are fraud and real.
plt.title("")
sns.countplot(df.fraudulent)
df.groupby('fraudulent').count()['title'].reset_index().sort_values(by='title',ascending=False)

**Filling NAs with empty string**

In [ ]:
#filling na with empty space
df.fillna(' ', inplace=True)

In [ ]:
df

#### Total Job posting by countries and Total fake job posting by countries

In [ ]:
#obtaining country name by splitting location
def split(location):
    l = location.split(',')
    return l[0]

df['country'] = df.location.apply(split)

In [ ]:
country = dict(df.country.value_counts()[:11])
del country[' ']
plt.figure(figsize=(8,6))
plt.title('No. of job postings country wise', size=20)
plt.bar(country.keys(), country.values())
plt.ylabel('No. of jobs', size=10)
plt.xlabel('Countries', size=10)

In [ ]:
#distribution of characters in description feature 

fig,(ax1,ax2)= plt.subplots(ncols=2, figsize=(17, 5), dpi=100)
length= df[df["fraudulent"]==1]['description'].str.len()
ax1.hist(length,bins = 20,color='orangered')
ax1.set_title('Fake Post')
length=df[df["fraudulent"]==0]['description'].str.len()
ax2.hist(length, bins = 20)
ax2.set_title('Real Post')
fig.suptitle('Characters in description')
plt.show()

In [ ]:
df.head(10)

In [ ]:
#heatmap fpr correlation of numeeric features 
from sklearn.preprocessing import LabelEncoder

df_encoded = df.copy()
for col in df_encoded.select_dtypes(include=['object']).columns:
    df_encoded[col] = LabelEncoder().fit_transform(df_encoded[col])

corr = df_encoded.corr()
plt.figure(figsize=(10,10))
sns.heatmap(corr, annot=True, cmap="coolwarm")
plt.show()

In [ ]:
#subsetting fake and real data 
fake=df[df.fraudulent==1]
#fake
real = df[df.fraudulent==0]

In [ ]:
#categorical plots to cheeck relationship of binary features with fradulent transaction 
sns.catplot(data=df,y='fraudulent', x='telecommuting', kind='bar')
sns.catplot(data=df, y='fraudulent', x='has_company_logo', kind='bar')
sns.catplot(data=df, y='fraudulent', x='has_questions', kind='bar')

**Observation**

The following graphs of three binary features show that ads of jobs involving telecommuting, do not have company logo and screening questions are more likely to be fake. Among the three features, the absence of company logo is the most indicative, with 16% of ads without company logo to be fake, while only 2% of ads with company logo are fraudulent.



In [ ]:
#barplot for visualization of data points in each countries for both fake jobs 
country = dict(fake.country.value_counts()[:11])
del country[' ']
plt.figure(figsize=(8,6))
plt.title('No. of fake job postings country wise', size=20)
plt.bar(country.keys(), country.values())
plt.ylabel('No. of fake jobs', size=10)
plt.xlabel('Countries', size=10)

In [ ]:
#barplot for visualization of data points in each countries for both real jobs 

country = dict(real.country.value_counts()[:11])
del country[' ']
plt.figure(figsize=(8,6))
plt.title('No. of real job postings country wise', size=20)
plt.bar(country.keys(), country.values(), color ='orange')
plt.ylabel('No. of real jobs', size=10)
plt.xlabel('Countries', size=10)

**Observation:**

It looks like United states has most job posting in the data compared to other countries and even the fake job posting are mostly in the United states and other countries has very less amount of fake job posting. 


#### Fake job postings by title.

In [ ]:
#barplot of job postings by title of job

titles = dict(fake.department.value_counts()[:11])
del titles[' ']
plt.figure(figsize=(8,6))
plt.title('No. of fake job postings department wise', size=20)
names = list(titles.keys())
values = list(titles.values())

plt.barh(range(len(titles)), values, tick_label=names)
plt.ylabel('Titles of job', size=10)
plt.xlabel('No of fake job', size=10)
plt.show()



### Difference in characters of description of fake jobs and real jobs

In [ ]:
fig,(ax1,ax2)= plt.subplots(ncols=2, figsize=(17, 5), dpi=100)
length= df[df["fraudulent"]==1]['description'].str.len()
ax1.hist(length,bins = 20,color='orangered')
ax1.set_title('Fake Post')
length=df[df["fraudulent"]==0]['description'].str.len()
ax2.hist(length, bins = 20)
ax2.set_title('Real Post')
fig.suptitle('Characters in description')
plt.show()

## Data Preparation

Merging seven text column which are significant on determining real and fakee jobs. The features “title”, “company profile”, “description”, “employment_type”, “required_experience” , “required education” and “industry” were combined into a single feature called “text” 


In [ ]:
#combining significant text features 
df['text']=df.title+' '+df.company_profile+' '+df.description+' '+df.employment_type+' '+df.required_experience+' '+df.required_education+' '+df.industry

In [ ]:
df

In [ ]:
#counting characters in text feature for each data points 
df['character_count'] = df['text'].str.len()


In [ ]:
df

In [ ]:
#frequency plot for distribution of words on text

plt.figure(figsize=(10,10))

df[df.fraudulent==0].character_count.plot(bins=35, kind='hist', color='blue', 
                                       label='Real', alpha=0.8)
df[df.fraudulent==1].character_count.plot(kind='hist', color='red', 
                                       label='Fake', alpha=0.8)
plt.legend()
plt.title('Frequency of the Words in Text Information ',size=20)
plt.xlabel("Character Count",size=10);

**The distribution of charaters in description of the fake and real post are similar. Some fake post reach to 6000 to 8000 characters.**

## Data Cleaning and Preprocessing

#### Selecting features relavant for text classification. 

In [ ]:
#subsetting necessary features
features = ['text', 'fraudulent']

In [ ]:
#making new data frame with relavant features 
df_new = df[features]
df_new

In [ ]:
df_new["text"][1]

**Removing HTML tags from the dataframe**

In [ ]:
#cleaning the html tags from the data 
for i in df.index:
    df_new.at[i, 'text'] = re.sub(r'https?://\S+', '', df_new.at[i, 'text'])
    

#testing if the code worked    
df_new["text"][1]

In [ ]:
df_new

### Functions for Cleaning

1) Removing HTML tags

2) Removing punctuations

3) Removing Stop words

4) Removing Numerical values

5) Case Normalization 

6) Lemmatization

7) Tokenization

In [ ]:
punctuation = set(punctuation)
import nltk
#nltk.download('wordnet')
from nltk.stem import WordNetLemmatizer

# speeds up comparison

def remove_punc(text):
    text_nonpunc ="".join([char for char in text if char not in punctuation])
    return text_nonpunc

# Define Tokenization Function
#To keep the emojis while tokenizing
RE_TOKEN = re.compile(r"""
                   ( [#]?[@\w'’\.\-\:]*\w     # words, hashtags and email addresses
                   | [:;<]\-?[\)\(3]          # coarse pattern for basic text emojis
                   | [\U0001F100-\U0001FFFF]  # coarse code range for unicode emojis
                   )
                  """, re.VERBOSE)


    
def tokenize(text):
    return text.split()

# Define Stop Words Remove Function
stopwords = set(nltk.corpus.stopwords.words('english'))

#function for removing stop words 
def remove_stop(text, stop_words = sw) :
     # modify this function to remove stopwords .split(' ')
    return(" ".join([word for word in text.split(' ') if word not in stop_words ]))
 
#function for removing  punctuation

def remove_punctuation(text, punct_set=punctuation) : 
    return("".join([ch for ch in text if ch not in punct_set]))


#function for lemmmatization

lemmatizer = WordNetLemmatizer()
def lemmatize_words(text):
    words = text.split()
    words = [lemmatizer.lemmatize(word,pos='v') for word in words]
    return ' '.join(words)

#function for reeemoving numeric data 

def remove_number(text):
    return (''.join([x for x in text if not x.isdigit()]))


# Define pipeline of lowering, no punctuation, tokenization, and removal of stopwords
pipeline1 = [str.lower,remove_stop,remove_punctuation,remove_number,lemmatize_words]


def prepare(text, pipeline): 
    tokens = text
    for transform in pipeline: tokens = transform(tokens)
    return tokens

### Removing Puctuations, Lower Casing and Removing Stop Words

In [ ]:
#applying cleaning pipeline to the text data
df_new['clean_text'] = df_new['text'].apply(prepare, pipeline = pipeline1)

df_new

In [ ]:
df_new['clean_text'][1]

In [ ]:
#applying cleaning pipeline to the text data

pipeline2= [tokenize]
df_new['token'] = df_new['clean_text'].apply(prepare, pipeline = pipeline2)

df_new

## Descriptive Statistics of Data

In [ ]:
#function for generating descriptive statistics 

def descriptive_stats(tokens, num_tokens = 5, verbose=True) :
    
    #filled na with whitespace to work with df and making sure it will
    #not be counted on frequent words
    tokens = [token for token in tokens if token not in ['', " "]]
    # Fill in the correct values here. 
    num_tokens =  len(tokens)
    num_unique_tokens = len(set(tokens)) #len(np.unique(tokens))
    lexical_diversity =  len(set(tokens)) / len(tokens) 
    num_characters =  sum([len(i) for i in tokens]) 
    
    if verbose :        
        print(f"There are {num_tokens} tokens in the data.")
        print(f"There are {num_unique_tokens} unique tokens in the data.")
        print(f"There are {num_characters} characters in the data.")
        print(f"The lexical diversity is {lexical_diversity:.3f} in the data.")
    
        # print the five most common tokens
        print(Counter(tokens).most_common(5))

        
    return([num_tokens, num_unique_tokens,
            lexical_diversity,
            num_characters])

In [ ]:
#descriptive Stats of fraudulent text
fraud_df = df_new[df_new['fraudulent']== 1]
token_fraud = []
for i in fraud_df['token']:
    token_fraud.extend(i)    
print('Descriptive Stats of fraudulent description:')
descriptive_stats(token_fraud)

In [ ]:
#descriptive Stats of non fraudulent text


nonfraud_df = df_new[df_new['fraudulent']== 0]
token_nonfraud = []
for i in nonfraud_df['token']:
    token_nonfraud.extend(i)    
print('Descriptive Stats of Non fraudulent description:')
descriptive_stats(token_nonfraud)

## Word clouds 

In [ ]:
df_new

In [ ]:
# Define function to create wordcloud:
from matplotlib import pyplot as plt

def wordcloud(word_freq, title=None, max_words=200, stopwords=None):

    wc = WordCloud(width=800, height=400, 
                   background_color= "black", colormap="Paired", 
                   max_font_size=150, max_words=max_words)
    
    # convert data frame into dict
    if type(word_freq) == pd.Series:
        counter = Counter(word_freq.fillna(0).to_dict())
    else:
        counter = word_freq

    # filter stop words in frequency counter
    if stopwords is not None:
        counter = {token:freq for (token, freq) in counter.items() 
                              if token not in stopwords}
    wc.generate_from_frequencies(counter)
 
    plt.title(title) 

    plt.imshow(wc, interpolation='bilinear')
    plt.axis("off")
    
    
def count_words(df, column='token', preprocess=None, min_freq=2):

    # process tokens and update counter
    def update(doc):
        tokens = doc if preprocess is None else preprocess(doc)
        counter.update(tokens)

        
   # create counter and run through all data
    counter = Counter()
    df[column].map(update)

    # transform counter into data frame
    freq_df = pd.DataFrame.from_dict(counter, orient='index', columns=['freq'])
    freq_df = freq_df.query('freq >= @min_freq')
    freq_df.index.name = 'token'
    
    return freq_df.sort_values('freq', ascending=False)

In [ ]:
from wordcloud import WordCloud 
#creating word cloud for both real and fake job posting text 

#fake posting 
plt.figure(figsize=(8,4))
fraud_wd= count_words(df_new[df_new['fraudulent']== 1])
wordcloud(fraud_wd['freq'], max_words= 50)
plt.title("Fraudulent Text", size = 18)

#real posting 
plt.figure(figsize=(8,4))
nonfraud_wd= count_words(df_new[df_new['fraudulent']== 0])
wordcloud(nonfraud_wd['freq'], max_words= 50)
plt.title("Non Fraudulent Text", size = 18)

## Sampling techniques

We have class imbalance problem.Therefore we will be comparing the model performances of original distribution with both under sampling and over sampling. 


## Modeling 

### Imbalanced  Data Set

In [ ]:
X=df_new.drop(['fraudulent','text', 'token'], axis=1)
y=df_new['fraudulent']

In [ ]:
type(X)

In [ ]:
X.shape

In [ ]:
y.shape

## Featuree Engineering 

1) BoW

2) TF-IDF

In [ ]:
from nltk.corpus import stopwords
stop_words_list = list(stopwords.words('english'))

tfidf_text_vectorizer = TfidfVectorizer(stop_words='english', min_df=5, max_df=0.7)

tfidf_text_vectors = tfidf_text_vectorizer.fit_transform(X['clean_text'])
print(tfidf_text_vectors.shape)

In [ ]:
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import CountVectorizer
stop_words_list = list(stopwords.words('english'))

count_text_vectorizer = CountVectorizer(stop_words=stop_words_list, min_df=5, max_df=0.7)
count_text_vectors = count_text_vectorizer.fit_transform(X['clean_text'])
print(count_text_vectors.shape)

### Train-Test Split

In [ ]:
X_train_tf, X_test_tf, y_train_tf, y_test_tf = train_test_split(tfidf_text_vectors,y, test_size=0.20,random_state=0,stratify=y)

X_train_bow, X_test_bow, y_train_bow, y_test_bow = train_test_split(count_text_vectors,y, test_size=0.20,random_state=0,stratify=y)

In [58]:
y_train_tf.value_counts()

0    13611
1      693
Name: fraudulent, dtype: int64

In [59]:
y_train_bow.value_counts()

0    13611
1      693
Name: fraudulent, dtype: int64

In [ ]:
#saving clean data on local machine  
df_new.to_csv(r'C:\Users\ADMIN\Documents\Jupyter\Fake Job Postings\clean_df.csv')